In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import pickle as pk
import resources.RES as res
from sklearn.preprocessing import minmax_scale as sk_minmax

In [2]:
# retrieve context
context_path = os.path.relpath("../input/testcase_context.obj") 
context_mesh = tm.load(context_path)

In [3]:
# retrieve lattice
env_lat_path = os.path.relpath("../data/testcase_lattice.csv")
envelope_lattice = tg.lattice_from_csv(env_lat_path)

In [4]:
# sunvectors
sunvectors = pk.load(open("../data/T2_sunvectors.pk", "rb"))

In [5]:
# create trimesh cuboids for computing intersections
def transform_mat(value):
    mat = np.identity(4)
    mat[:3,-1] = np.array(value)
    return mat

In [6]:
# voxel cuboid meshes
vox_cuboids = [tm.creation.box(envelope_lattice.unit, transform=transform_mat(ct)) for ct in envelope_lattice.centroids]

# number of faces per voxel
f_numb = len(vox_cuboids[0].faces)

# combine voxels into one mesh
combined_voxels = tm.util.concatenate(vox_cuboids)

In [7]:
# combine voxels and context into one mesh
combined_meshes = tm.util.concatenate(combined_voxels, context_mesh)

# PV positions on roof
unit = envelope_lattice.unit[0]
dist = unit/2 # distance from centroid to top of voxel

transfrm = [0,0,dist] 
ctr = envelope_lattice.centroids

newctr = ctr + transfrm  # move z-value of centroids for placement/calculation of PV

PV_ray_ctr = np.tile(-sunvectors, [len(newctr),1]) # PV ray for each centroid
PV_ctr_ray = np.tile(newctr, [1, len(sunvectors)]).reshape(-1, 3) # PV centroid for each ray

In [8]:
# intersection of rays from voxel centroids to sun objects with all voxel faces
face_id, ray_id = combined_meshes.ray.intersects_id(ray_origins=PV_ctr_ray, ray_directions=PV_ray_ctr, multiple_hits=True)

In [9]:
G1, U1 = res.construct_graph(sunvectors, face_id, ray_id, envelope_lattice, f_numb)

In [10]:
G1.shape, U1.shape

((315, 315, 324), (315, 324))

In [11]:
# save interedependencies for later use
#pk.dump(G1, open("../data/T3_SolG1.pk", "wb")) # too big

# save obstructions for later use
#pk.dump(U1, open("../data/T3_SolU1.pk", "wb"))